In [1]:
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from datetime import datetime

In [34]:
lengths_from_file = {}
with open('dataframe_lengths.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        dataframe_name, length = row
        lengths_from_file[dataframe_name] = int(length)

In [3]:
lengths_from_file

In [4]:
lay1 = pd.read_csv('ABC_layout_1.csv')
del lay1['Last Name']
lay1c = lay1.copy().loc[lengths_from_file['lay1']:]
# lay1

In [5]:
lay2 = pd.read_csv('PQR_layout_2.csv')
lay2.drop(columns=['Unnamed: 0'], inplace=True)
lay2c = lay2.copy().iloc[lengths_from_file['lay2']:]

In [6]:
lay3 = pd.read_csv('XYZ_layout_3.csv')
lay3c= lay3.copy().iloc[lengths_from_file['lay3']:]

In [7]:
lay4 = pd.read_csv('KLM_layout_4.csv')
lay4.drop(columns=['Unnamed: 0'], inplace=True)
lay4c = lay4.copy().iloc[lengths_from_file['lay4']:]

In [8]:
lay5 = pd.read_csv('DOTM_layout_5.csv')
lay5c = lay5.copy().iloc[lengths_from_file['lay5']:]

In [9]:
dataframes = [lay1,lay2,lay3,lay4,lay5]
lengths = {f'lay{i+1}': len(dataframe) for i, dataframe in enumerate(dataframes)}
with open('dataframe_lengths.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['DataFrame', 'Length'])
    for name, length in lengths.items():
        writer.writerow([name, length])

In [10]:
def merge_columns(df):
    # List columns ending with _x and _y
    merge_cols = [col[:-2] for col in df.columns if col.endswith('_x')]
    
    for col in merge_cols:
        col_x = f'{col}_x'
        col_y = f'{col}_y'
        df[col] = df[col_x].combine_first(df[col_y])
    
    # Drop _x and _y columns
    drop_cols = [col for col in df.columns if col.endswith('_x') or col.endswith('_y')]
    df.drop(drop_cols, axis=1, inplace=True)
    
    return df

In [11]:
def combine_columns(df, column_names, new_column_name):
    if not all(col in df.columns for col in column_names):
        raise ValueError("Some column names do not exist in the DataFrame")
    
    def transform_date(date):
        if pd.isna(date):
            return ''
        f_date = date.replace('-', '')
        return f_date

    def transform_address(address):
        if pd.isna(address):
            return ''
        address = address.replace(',', '').replace(' ', '').replace('Nepal', '')
        return address

    def transform_name(name):
        if pd.isna(name):
            return ''
        f_name = name.replace(' ', '')
        return f_name
    
    df.columns = df.columns.str.lower()
    columns = [item.lower() for item in column_names]
    for col in columns:
        if 'date' in col:
            df[col] = df[col].apply(transform_date)
        elif 'address' in col:
            df[col] = df[col].apply(transform_address)
        elif 'name' in col:
            df[col] = df[col].apply(transform_name)
    df[new_column_name] = df[columns].astype(str).agg(' '.join, axis=1)
    return df

In [12]:
df = lay1c.copy()
df = combine_columns(lay1.copy(),['Name','Father Name','Date of Birth'], 'combined')
# df

In [13]:
df1 = lay2c.iloc[lengths_from_file['lay2']:].copy()
df1 = combine_columns(lay2.copy(),['Name','Father Name','Date of Birth'], 'combined')

In [14]:
def combine(odf, odf1, df, df1, Pid1, Pid2):
    vectorizer = TfidfVectorizer()
    df_tfidf_matrix = vectorizer.fit_transform(df['combined'])
    df1_tfidf_matrix = vectorizer.transform(df1['combined'])
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(df_tfidf_matrix)
    distances, indices = knn.kneighbors(df1_tfidf_matrix, n_neighbors=1)
    
    matches = []
    for i in range(len(df1)):
        match_id = df.iloc[indices[i][0]][Pid1.lower()]
        lookup_id = df1.iloc[i][Pid2.lower()]
        distance = distances[i][0]
        if distance < 0.5:
            matches.append((lookup_id, match_id, distance))
        else:
            matches.append((lookup_id, None, distance))
    
    matches_df = pd.DataFrame(matches, columns=['lookup_id', 'matched_id', 'distance'])
    
    odf1 = odf1.merge(matches_df[['matched_id', 'lookup_id']], left_on=Pid2, right_on='lookup_id', how='left')
    odf1.drop(columns=['lookup_id'], inplace=True)
    
    merged_df = odf1.merge(odf, left_on='matched_id', right_on=Pid1, how='left')
    merged_df.drop(columns=['matched_id'], inplace=True)
    
    return merged_df

In [15]:
c_df1 = combine(lay1c,lay2c,df,df1,'Customer Code','Customer ID')

In [16]:
c_df1.drop(columns=['Customer ID'], inplace=True)

In [17]:
c_df1 = merge_columns(c_df1.copy())


In [18]:
c_df1

In [19]:
copy_c_df1 = c_df1.copy()
copy_c_df1 = combine_columns(copy_c_df1, ['Name','Father Name','Date of Birth'], 'combined')

In [20]:
df3 = combine_columns(lay3c.copy(), ['Name','Father Name', 'Date of Birth'], 'combined')
c_df2 = combine(c_df1,lay3c,copy_c_df1,df3,'Customer Code','votersID')

In [21]:
c_df2 = merge_columns(c_df2.copy())

In [22]:
copy_c_df2 = c_df2.copy()
copy_c_df2 = combine_columns(copy_c_df2, ['Name','Father Name','Date of Birth'], 'combined')

In [23]:
df4 = combine_columns(lay5c.copy(),['Name','Father Name','Date of Birth'], 'combined')
c_df3 = combine(c_df2,lay5c,copy_c_df2,df4,'Customer Code','License Number')

In [24]:
c_df3 = merge_columns(c_df3.copy())

In [25]:
copy_c_df3 = c_df3.copy()
copy_c_df3 = combine_columns(copy_c_df3, ['Name','Father Name','Date of Birth'], 'combined')

In [26]:
df5 = combine_columns(lay4c.copy(),['Name','Father Name','Date of Birth'], 'combined')
c_df4 = combine(c_df3,lay4c,copy_c_df3,df5,'Customer Code','Customer ID')

In [27]:
c_df4 = merge_columns(c_df4.copy())

In [28]:
final_df = c_df4[['Name','Date of Birth','Gender','Father Name','Mother Name','Permanent Address','Mobile Number','Temporary Address','Citizenship Number','License Number','Blood Group']]
final_df['Updated Date'] = datetime.today().strftime('%Y-%m-%d')

In [29]:
final_df

In [30]:
import string
import random

def generate_unique_id(df):

    if 'uid' not in df.columns:
        df['uid'] = ''

    for i in range(len(df)):
        if df.loc[i, 'uid'] == '' or pd.isna(df.loc[i, 'uid']):
            chars = string.ascii_uppercase + string.digits
            uid = ''.join(random.choice(chars) for _ in range(10))
            while uid in df['uid'].unique():
                uid = ''.join(random.choice(chars) for _ in range(10))
            df.loc[i, 'uid'] = uid

    return df

In [31]:
final_df = generate_unique_id(final_df.copy())
final_df

In [30]:
new_data = {
    'ID':'112233',
    'Name': 'Alice Johnson',
    'Date of Birth': '1992-07-07',
    'Father Name': 'Dinesh Johnson',
    'Gender': 'F',
    'Email': 'alice@example.com',
    'Permanent Address': '789 Maple St'
}

new_df = pd.DataFrame([new_data])

In [31]:
data = combine_columns(new_df.copy(),['Name','Father Name','Date of Birth'],'combined')
data

In [32]:
df8 = final_df.copy()
df8 = combine_columns(df8,['Name','Father Name','Date of Birth'],'combined')

In [33]:
nnew_df = combine(final_df,new_df,df8,data,'Citizenship Number','ID')

In [34]:
nnew_df

In [35]:
nnew_df['Date of Birth'] = nnew_df['Date of Birth_x'].combine_first(nnew_df['Date of Birth_y'])
nnew_df['Name'] = nnew_df['Name_x'].combine_first(nnew_df['Name_y'])
nnew_df['Father Name'] = nnew_df['Father Name_x'].combine_first(nnew_df['Father Name_y'])
nnew_df['Permanent Address'] = nnew_df['Permanent Address_x'].combine_first(nnew_df['Permanent Address_y'])
nnew_df['Gender'] = nnew_df['Gender_x'].combine_first(nnew_df['Gender_y'])
nnew_df.drop(
    ['Date of Birth_x', 'Date of Birth_y', 'Name_x', 'Name_y', 'Father Name_x', 'Father Name_y','Gender_x','Gender_y','Permanent Address_x','Permanent Address_y'], axis=1, inplace=True)

In [36]:
nnew_df.drop('ID', axis=1, inplace=True)

In [37]:
final_df

In [38]:
concatenated_df = pd.concat([final_df,nnew_df],ignore_index=True)

In [39]:
concatenated_df = generate_unique_id(concatenated_df.copy())
concatenated_df

In [35]:
lay5 = pd.read_csv('DOTM_layout_5.csv')
lay5c = lay5.copy().iloc[lengths_from_file['lay5']:]

In [36]:
lay5c

In [37]:
copy_lay5c = lay5c.copy()
copy_lay5c = combine_columns(copy_lay5c, ['Name','Father Name','Date of Birth'], 'combined')
copy_lay5c

In [38]:
df9 = final_df.copy()
df9 = combine_columns(df9,['Name','Father Name','Date of Birth'],'combined')
df9

In [39]:
new_df = combine(final_df,lay5c,df9,copy_lay5c,'Citizenship Number','License Number')

In [40]:
new_df

In [41]:
new_df = merge_columns(new_df.copy())

In [42]:
new_df

In [43]:
concatenated_df = pd.concat([final_df,new_df],ignore_index=True)

In [44]:
concatenated_df = generate_unique_id(concatenated_df.copy())
concatenated_df